<a href="https://colab.research.google.com/github/rhiats/mimicIII/blob/main/Analysis_of_Patients_with_Type_2_Diabetes_in_ICU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stratification Analysis of Patients with Type 2 Diabetes in ICU**

Person diagnosed with Diabetes.

Type 2 Diabetic ICD9 Codes (250)/ Admissions - 'diabet...'.

- Patients - distribution of age
- Admissions- breakdown of insurance type. (one hot encode)
- Admissions - Probability of having diabetes and being male/female (one hot encode)
- Prescriptions  - Top 3 drugs (one hot encode)
- Prescriptions - Distribtuion number of drugs per patient
- ICUSTAYS - Distribution of time in ICU
- Open (DIAGNOSES_ICD.csv.gz)

http://www.icd9data.com/2014/Volume1/240-279/249-259/250/250.00.htm

https://physionet.org/content/mimic3-carevue/1.4/

https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [2]:
import pandas as pd

In [25]:
df=pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/ADMISSIONS.csv.gz', compression='gzip')

def diabetetesFind(x):
  """
    Find patients diagnosed with Diabetes.
    @p: pandas series 'diagnosis'
    @r: dataframe with diabetic patients
  """
  x=str(x)
  x=x.lower()
  return x

df['diagnosis_lower']=df['diagnosis'].apply(diabetetesFind)
df_diab=df[df['diagnosis_lower'].str.contains(pat = 'diabet')]

num_patients=df_diab['subject_id'].nunique()
'There are {} patients in the ICU diagnosed diabetes.'.format(num_patients)

'There are 131 patients in the ICU diagnosed diabetes.'

In [2]:
#Year
#Age
#Sex

**PCA**

**Scree Plot**

Select Feature 1

**Interpret the Weights of the First Component**

**Project each patient on the First Component to Calculate their Severity Score**

In the future, patients can be projected on the component to determine their risk for developing Diabetes based on pre-existing conditions.